In [1]:
import pandas as pd 
import numpy as np 
import sklearn.feature_extraction.text as ft 
import nltk
import sys 
from tqdm.notebook import tqdm
import shap
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
sys.path.insert(0, '../')
import src.preprocessing.text_preprocessing as tp
import src.preprocessing.feature_extraction.text.filtering as filter
%load_ext autoreload
%autoreload 2
tqdm.pandas()

In [4]:
df = pd.read_csv('../data/enron/enron_spam_data.csv', sep=',')
df = df.fillna('')
df = df.astype('str')
df['Text'] = df.apply(lambda x: x['Subject'] + ', ' + x['Message'], axis=1)
df['Label'] = np.where(df['Spam/Ham'].values == 'ham', 0, 1)
df['Text'] = df['Text'].progress_apply(tp.normalize_text)

  0%|          | 0/33716 [00:00<?, ?it/s]

In [6]:
import pickle

with open('../words_per_class_dict.pkl', 'rb') as file:
    words_per_class_dict = pickle.load(file)

In [7]:
words_per_class_dict.keys()

dict_keys(['tfidf', 'term_strength', 'mi', 'chi2', 'shap'])

In [8]:
count_vectorizer = ft.CountVectorizer()
count_vectorizer.fit(df['Text'])
vocabulary = count_vectorizer.get_feature_names_out()

In [17]:
test_df = pd.DataFrame(index=vocabulary, columns=['tfidf_0', 'tfidf_1', 'term_strength', 'mi', 'chi2', 'shap'])

In [22]:
test_df.loc[words_per_class_dict['mi'][0], 'mi'] = 1
test_df.loc[words_per_class_dict['tfidf'][0], 'tfidf_0'] = 1
test_df.loc[words_per_class_dict['tfidf'][1], 'tfidf_1'] = 1
test_df.loc[words_per_class_dict['term_strength'][0], 'term_strength'] = 1
test_df.loc[words_per_class_dict['chi2'][0], 'chi2'] = 1
test_df.loc[words_per_class_dict['shap'][0], 'shap'] = 1


In [24]:
test_df = test_df.fillna(0)

In [26]:
test_df.corr()

,tfidf_0,tfidf_1,term_strength,mi,chi2,shap
tfidf_0,1.000000,0.397905,0.821194,0.573907,0.543387,0.107588
tfidf_1,0.397905,1.000000,0.430798,0.560750,0.518259,0.106941
term_strength,0.821194,0.430798,1.000000,0.590947,0.539181,0.113196
mi,0.573907,0.560750,0.590947,1.000000,0.695987,0.109745
chi2,0.543387,0.518259,0.539181,0.695987,1.000000,0.101333
shap,0.107588,0.106941,0.113196,0.109745,0.101333,1.000000


In [29]:
from sklearn.metrics import jaccard_score

In [42]:
jaccard_score(test_df['mi'], test_df['tfidf_1'])

0.4211610886093939